In [1]:
import polars as pl
import lightgbm as lgb
import numpy as np

In [2]:
bst = lgb.Booster(model_file = '/boosters/25M.txt')
test = pl.read_parquet(f"/sleep/test_selected.parquet")

In [ ]:
preds_proba = bst.predict(test.drop('state', 'series_id', 'step', 'timestamp'))
preds = np.argmax(preds_proba, axis = 1)

In [4]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test.select('state'), preds))

0.8274695019258088


In [142]:
# Find indices where the value goes from 0 to 1
onset_events = []
wake_events = []

for event in ['onset','wake']:
    
    if event == 'onset':
        transitions = np.where(np.diff(preds) == 1)[0]
        indices = [idx for idx in transitions if np.all(preds[idx+1:idx+10] == 1) and np.all(preds[idx-1:idx-10] == 0)]
        #indices = transitions
    else:
        transitions = np.where(np.diff(preds) == -1)[0]
        indices = [idx for idx in transitions if np.all(preds[idx+1:idx+10] == 0) and np.all(preds[idx-1:idx-10] == 1)]
        #indices = transitions 
    
    windows = []
    frame_start = indices[0]
    current_frame = []
    
    for index in indices:
        if index-frame_start > 5760: #6 hours between two sleeping periods
            windows.append(current_frame)
            current_frame = [index]
            frame_start = index
        else:
            current_frame.append(index) 
    for frames in windows:
        
        frame_preds = preds_proba[frames]
        column_difference = np.abs(frame_preds[:, 0] - frame_preds[:, 1]) 
        """ 
        column_difference = []
        for frame in frames:
            column_difference.append(np.abs(np.mean(preds_proba[frame-10:frame, 0]) - np.mean(preds_proba[frame:frame+10, 1])))
        """
        if event == 'onset':
            onset_events.append(frames[np.argmax(column_difference)])
        else:
            wake_events.append(frames[np.argmax(column_difference)])

 
users_id = list(range(265,277))
out_dict = {'row_id':[], 'series_id': [], 'step': [], 'event': [], 'score': []}
wake_index = 0
onset_index = 0
row_id = 0
for id in users_id:
    max_step = test.filter(pl.col('series_id') == id).max().select('step').item() 
    if max_step is None:
        continue
    while onset_index < len(onset_events) and wake_index < len(wake_events) and wake_events[wake_index] <= max_step and onset_events[onset_index] <= max_step :       
        #Case where we any index is over the threshold
        
        if wake_events[wake_index] > max_step: 
            onset_index += 1
        elif onset_events[onset_index] > max_step:
            wake_index += 1
        
        #1440-8640 range where accept that person slept
        elif wake_events[wake_index]-onset_events[onset_index] in range(720,8641):
            out_dict['row_id'].extend([row_id, row_id+1])
            row_id += 2
            out_dict['series_id'].extend([id, id])
            out_dict['step'].extend([onset_events[onset_index],wake_events[wake_index]])
            out_dict['event'].extend(['onset', 'wakeup'])
            onset_score = min(1,np.abs(preds_proba[onset_events[onset_index],0]-preds_proba[onset_events[onset_index],1]))
            wakeup_score = min(1,np.abs(preds_proba[wake_events[wake_index],0]-preds_proba[wake_events[wake_index],1]))
            out_dict['score'].extend([onset_score,wakeup_score])
            wake_index += 1
            onset_index += 1

        elif wake_events[wake_index]-onset_events[onset_index] < 720:
            wake_index += 1

        else:
            onset_index += 1
            
    #Correct the indices
    wake_events = [step-max_step for step in wake_events]
    onset_events = [step-max_step for step in onset_events]

out = pl.from_dict(out_dict)
out.write_csv('events_pred.csv')

In [143]:
print(len(out))
from apmetric import score
column_names = {
    'series_id_column_name': 'series_id',
    'time_column_name': 'step',
    'event_column_name': 'event',
    'score_column_name': 'score',
}

tolerances = {
    'onset': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360], 
    'wakeup': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360]
}

df_copy = df.clone()
df_copy = df_copy.with_columns(pl.col('event').map_dict({1: 'onset', 2: 'wakeup'}))
df_copy = df_copy.filter(pl.col('series_id').is_in(users_id)).drop(['night','timestamp'])
print(score(df_copy.to_pandas(),out.drop('row_id').to_pandas(),tolerances, **column_names))

342
0.22368602606396532


In [101]:
df2 = pl.scan_csv('events_compressed.csv')
unique = df2.unique(subset=['series_id'], maintain_order = True).select('series_id').collect().to_numpy().flatten()
print(unique.flatten())

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  55
  56  57  58  59  60  61  62  63  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 141 142 143 144 145 146 147
 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165
 166 167 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184
 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 202 203
 204 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222
 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240
 241 243 244 245 246 247 248 249 250 251 252 253 25